In [1]:
import os
import numpy as np
import sys
import scatlib
from netCDF4 import Dataset

In [7]:
# iscat = input("enter 1 for Ku-Scat; 2 for Ka-Scat: ")


raw_path = 'testing/Raw_Canonical/scan/'
proc_path = 'testing/Raw_nc/'

files = os.listdir(raw_path)
files

for filename in files:

    if 'KA-SCAT' in filename:
        iscat = 2
    elif 'KU-SCAT' in filename:
        iscat = 1
    else:
        iscat = 0

    if iscat > 0:
            
        configvars = scatlib.read_configuration(iscat)
        
        configvars["raw_data_path"] = raw_path
        configvars["processed_data_path"] = proc_path
        # configvars["raw_data_path"] = sys.argv[1]
        # configvars["processed_data_path"] = sys.argv[2]
        
        instrument_name = configvars["instrument_name"]
        calfile = configvars["calfile"]    #pass this as plain variable since it may get modified if creating a new calfile
        raw_data_path = configvars["raw_data_path"]
        processed_data_path = configvars["processed_data_path"]
    
        try:
            ## READ_HEADER
            
            header_output = scatlib.read_header(raw_path+filename, raw_data_path, configvars)
    
            # rcw get the reference_calibration_loop_power_file         
            reference_calibration_loop_power_file = 10**(header_output[1]["cal_peak_dbm"]/10.)  
    
            ## READ_RAW
    
            raw_output =  scatlib.read_raw(configvars, header_output[0],raw_path+filename,
                                                     header_output[3], header_output[4])
    
            write_nc(configvars,filename,raw_output,header_output)
            print('SUCCESSFULLY WRITTEN')
    
        except Exception as e:
            print(filename)
            print(e)

Reading SCAT_IDL_config....
Reading Ku-SCAT_IDL_config
AD clock frequency (MHz):  20.0
AD trig delay (ns):  4000
chirp bandwidth (MHz):  6000.0
chirp center frequency (MHz):  15000.0
chirp amplitude dbm:  0.0
chirp width (ms):  2.0
decimation:  32
decimation mode:  0
frame delay (ms):  500.0
number of time samples per sweep (n_gates):  1250
pulse repetition period (ms):  2.2
nominal range resolution (c_light/2/chirp_bandwidth, m):  0.025
server state, 0=idle# 1=record:  1
chirp bandwidth (MHz):  6000.0
antenna beamwidth, (deg):  16.9
corner reflector radar cross section (square meters):  16.77
pedestal_max_speed (deg/s):  8.0
pedestal height (m):  1.09
Meta Header Size 146
start time:  1600055899  seconds
nano-seconds:  194124674
reading block number:  0  time:  Mon Sep 14 3:58:19  2020
reading block number:  100  time:  Mon Sep 14 3:59:50  2020
reading block number:  200  time:  Mon Sep 14 3:59:52  2020
reading block number:  300  time:  Mon Sep 14 4:0:38  2020
reading block number:  

In [13]:
d = Dataset('/home/robbie/kukapy_RM/testing/Raw_nc/KU-SCAT-20200914-035819.nc')
d['counts'].shape

(782, 6, 1250)